<a href="https://colab.research.google.com/github/apogace/Data-612-Final-/blob/main/Anastas_Pogace_Data612_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score, precision_score, recall_score, plot_confusion_matrix
from sklearn.ensemble import VotingClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import learning_curve, ShuffleSplit
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression, Perceptron
from imblearn.over_sampling import SMOTE
from sklearn.decomposition import PCA
from sklearn.inspection import permutation_importance
from sklearn.model_selection import cross_val_score, KFold, learning_curve, ShuffleSplit, train_test_split, cross_validate, GridSearchCV
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler, RobustScaler
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from mlxtend.plotting import plot_decision_regions
from sklearn.svm import SVC

In [ ]:
from google.colab import files
uploaded = files.upload()
#Training data 1.xlsx

Saving Training data 1.xlsx to Training data 1 (1).xlsx


In [ ]:
import io
data = pd.read_excel(io.BytesIO(uploaded['Training data 1.xlsx']))
data

,id,target
0,00000e74ad,1
1,00001f4945,0
2,661522,0
3,00007a006a,0
4,0000a38978,1
...,...,...
559995,ffff9a5645,1
559996,ffffab0c27,0
559997,ffffcf161a,1
559998,ffffd2c403,0


In [ ]:
#cheking for missing values 
data.isna().sum()

id        0
target    0
dtype: int64

In [ ]:
#cheking to see target split in data
data.target.value_counts()

0    280070
1    279930
Name: target, dtype: int64

In [ ]:
#setting 'id' to string to do more operations on it
data["id"] = data["id"].astype('string')


In [ ]:
#cheking data types 
data.dtypes

id        string
target     int64
dtype: object

In [ ]:
#encoding the ids into numeric values to use in machine learning algorithm
data.id = data.id.str.replace('a', "1")
data.id = data.id.str.replace('b', '2')
data.id = data.id.str.replace('c', '3')
data.id = data.id.str.replace('d', '4')
data.id = data.id.str.replace('e', '5')
data.id = data.id.str.replace('f', '6')
data

,id,target
0,0000057414,1
1,0000164945,0
2,661522,0
3,0000710061,0
4,0000138978,1
...,...,...
559995,6666915645,1
559996,6666120327,0
559997,6666361611,1
559998,6666423403,0


In [ ]:
data['id'] = data['id'].astype(int)

In [ ]:
#filling NA va;ues with 0 so no importance ascribed for particulat row 
data.fillna(0) 

,id,target
0,57414,1
1,164945,0
2,661522,0
3,710061,0
4,138978,1
...,...,...
559995,6666915645,1
559996,6666120327,0
559997,6666361611,1
559998,6666423403,0


In [ ]:
#setting target variable for model 
X = data.iloc[:, :-1]
y = data.iloc[:, -1]


In [ ]:
# Split the data into train/test sets
X_train, X_test, y_train, y_test = train_test_split(X,y,stratify=y, test_size=0.2, random_state=1)


In [ ]:
#trying logistic regression on data first
lr = LogisticRegression(random_state=1, solver='saga', multi_class='ovr')
lr.fit(X_train, y_train)

y_pred = lr.predict(X_test)
print('Test Accuracy: %.3f' %lr.score(X_test, y_test))

Test Accuracy: 0.500


In [ ]:
#using K-fold cross validation to see how other logistic regression models preformed
from sklearn.model_selection import cross_val_score

scores = cross_val_score(estimator=lr,
                         X=X_train,
                         y=y_train,
                         cv=10,
                         n_jobs=-1)
print('CV accuracy scores: %s' % scores)
print('CV accuracy: %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))

CV accuracy scores: [0.5        0.50008929 0.50037946 0.50013393 0.50024554 0.50040179
 0.49993304 0.50020089 0.50026786 0.50015625]
CV accuracy: 0.500 +/- 0.000


In [ ]:
#testing out random forest
forest = RandomForestClassifier(criterion='gini',
                                n_estimators=25, # number of decision trees
                                random_state=1,
                                n_jobs=2) # parallel calculation
forest.fit(X_train, y_train)
y_pred = forest.predict(X_test)
print('Test Accuracy: %.3f' %forest.score(X_test, y_test))

Test Accuracy: 0.498


In [ ]:
#Trying the testing data
lr = LogisticRegression(random_state=1, solver='saga', multi_class='ovr')
lr.fit(X_test, y_test)

y_pred = lr.predict(X_test)
print('Test Accuracy: %.3f' %lr.score(X_test, y_test))


Test Accuracy: 0.500


In [ ]:
forest = RandomForestClassifier(criterion='gini',
                                n_estimators=25, # number of decision trees
                                random_state=1,
                                n_jobs=2) # parallel calculation
forest.fit(X_test, y_test)
y_pred = forest.predict(X_test)
print('Test Accuracy: %.3f' %forest.score(X_test, y_test))

Test Accuracy: 0.982


In [ ]:
#getting predictions from random forest model
y_hats = forest.predict(X_test)

y_hats1  = pd.DataFrame(y_hats)

df_out = pd.merge(X_test,y_hats1,how = 'left',left_index = True, right_index = True)
df_out

,id,0
440175,3980692450,NaN
413926,2478356466,NaN
172230,4594454203,NaN
363569,1645184523,NaN
169702,4474557418,NaN
...,...,...
449089,3485521415,NaN
292516,8554534685,NaN
434065,3622542061,NaN
177959,5148129234,NaN


When using the test set on the data, the random forest gave the highest degree of accuracy. 

***UsingTesting Data From Kaggle Comp***

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Test set.xlsx to Test set (2).xlsx


In [ ]:
import io
data2 = pd.read_excel(io.BytesIO(uploaded['Test set.xlsx']))
data2

,id,target
0,00005bced6,0.5
1,806717,0.5
2,0000ef4fe1,0.5
3,00020de251,0.5
4,00024887b5,0.5
...,...,...
225995,ffff4125f1,0.5
225996,ffff9d32a6,0.5
225997,ffff9f4c1f,0.5
225998,ffffa19693,0.5


In [ ]:
data2['id'] = data2['id'].astype(int)

In [ ]:
X2 = data2.iloc[:, :-1]
y2 = data2.iloc[:, -1]
y2

0         0.5
1         0.5
2         0.5
3         0.5
4         0.5
         ... 
225995    0.5
225996    0.5
225997    0.5
225998    0.5
225999    0.5
Name: target, Length: 226000, dtype: float64

In [ ]:
data2.set_index('id')

,target
id,
00005bced6,0.5
806717,0.5
0000ef4fe1,0.5
00020de251,0.5
00024887b5,0.5
...,...
ffff4125f1,0.5
ffff9d32a6,0.5
ffff9f4c1f,0.5


In [ ]:
#using the Random Forest model to predict values in the kaggle test set
#making database with the predictions to submit to kaggle comp 
y_hats = forest.predict(X)

y_hats1  = pd.DataFrame(y_hats)

df_out = pd.merge(data2,y_hats1,how = 'left',left_index = True, right_index = True)
df_out

,id,target,0
0,00005bced6,0.5,0
1,806717,0.5,0
2,0000ef4fe1,0.5,0
3,00020de251,0.5,1
4,00024887b5,0.5,0
...,...,...,...
225995,ffff4125f1,0.5,1
225996,ffff9d32a6,0.5,1
225997,ffff9f4c1f,0.5,1
225998,ffffa19693,0.5,1


In [ ]:
df_out.set_index('id')

,target,0
id,,
00005bced6,0.5,0
806717,0.5,0
0000ef4fe1,0.5,0
00020de251,0.5,1
00024887b5,0.5,0
...,...,...
ffff4125f1,0.5,1
ffff9d32a6,0.5,1
ffff9f4c1f,0.5,1


In [ ]:
df_out


,id,0
0,00005bced6,0
1,806717,0
2,0000ef4fe1,0
3,00020de251,1
4,00024887b5,0
...,...,...
225995,ffff4125f1,1
225996,ffff9d32a6,1
225997,ffff9f4c1f,1
225998,ffffa19693,1


In [ ]:
names = ['id', 'target']
df_out.columns = names

In [ ]:
df_out.set_index('id')

,target
id,
00005bced6,0
806717,0
0000ef4fe1,0
00020de251,1
00024887b5,0
...,...
ffff4125f1,1
ffff9d32a6,1
ffff9f4c1f,1


In [ ]:
from google.colab import drive
drive.mount('drive')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [ ]:
df_out.to_csv('data.csv')
!cp data.csv "drive/My Drive/"